In [1]:
# pip install fredapi

In [2]:
import yfinance as yf
import pandas as pd
import requests
from fredapi import Fred
import glob

# Dataset

## Y-Finance

In [3]:
# Define the ticker symbol
ticker_symbol = "AAPL"

# Create a Ticker object
ticker = yf.Ticker(ticker_symbol)

# Fetch historical market data
#period = ['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max']
historical_data = ticker.history(period="5y") 
historical_data.reset_index(inplace=True) 
historical_data['Date'] = historical_data['Date'].dt.date
historical_data.head(5)


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2019-10-28,59.853616,60.296314,59.684279,60.247932,96572800,0.0,0.0
1,2019-10-29,60.228575,60.417265,58.680346,58.854519,142839600,0.0,0.0
2,2019-10-30,59.210137,59.340771,58.351356,58.847271,124522000,0.0,0.0
3,2019-10-31,59.810074,60.276961,57.395800,60.177776,139162000,0.0,0.0
4,2019-11-01,60.366473,61.912284,60.274549,61.885677,151125200,0.0,0.0


In [4]:
historical_data.columns = historical_data.columns.str.lower()
historical_data.columns = historical_data.columns.str.replace(' ', '_')
historical_data.head(2)

,date,open,high,low,close,volume,dividends,stock_splits
0,2019-10-28,59.853616,60.296314,59.684279,60.247932,96572800,0.0,0.0
1,2019-10-29,60.228575,60.417265,58.680346,58.854519,142839600,0.0,0.0


In [5]:
historical_data.to_csv('data/formatted_datasets/historical_data.csv')

## Fama-French 5 factors

1. **Mkt-RF**: represents the additional return that investors require for taking on the risk of investing in the stock market rather than in a risk-free asset.
- Formula: `Mkt-RF = Rm - RF` 

     where `Rm`: Expected return on stock market, `RF`: risk-free rate
- Mkt-RF can be negative, where return on risk-free asset (government bonds,..) is higher than the return on stock market.
- Example: If the stock market return is 3.61% and the risk-free rate is 2%, then the market risk premium is 1.61%.
2. **SMB**: size premium, excess return of small-cap stocks over large-cap stocks. The idea behind SMB is that historically, smaller companies (small-cap stocks) tend to outperform larger companies (large-cap stocks), although this comes with higher risk.
- Formula: `SMB = Small - Big` 

     where `Small`: return on small-cap stocks, `Big`: return on large-cap stocks
- Example: There are 2 portfolios: **Small-Cap** Portfolio: Composed of smaller companies, such as emerging tech firms or small biotech companies. **Large-Cap** Portfolio: Composed of well-known, established companies like Apple, Microsoft, or Johnson & Johnson. Over a particular year: The Small-Cap Portfolio returns 12%, The Large-Cap Portfolio returns 8%. The size premium would be 4% (12% - 8%).
3. **HML**: value premium, which is the difference in returns between stocks with high book-to-market ratios (value stocks) and stocks with low book-to-market ratios (growth stocks). The idea behind HML is that historically, value stocks tend to outperform growth stocks.
- More explanation:

    +  Value Stocks (High Book-to-Market): These are companies whose stock prices are low relative to their book value (assets minus liabilities). They are considered "undervalued" by the market, and they may be mature companies with steady, but slow, growth. Investors expect these stocks to eventually revert to a higher price as their fundamental value becomes recognized.

    +  Growth Stocks (Low Book-to-Market): These are companies whose stock prices are high relative to their book value. These stocks are considered "overvalued" based on their fundamentals but are expected to grow rapidly in the future, justifying their higher prices.
- General Formula: `HML = High - Low` 

     where `High`: return on value stocks, `Low`: return on growth stocks
- Formula in FF:  is the average return on the two value portfolios minus the average return on the two growth portfolios
 
 `HML = 1/2 * (Small Value + Big Value) - 1/2 * (Small Growth + Big Growth)`
4. **RMW**: profitability premium, which is the difference in returns between stocks of companies with high operating profitability and stocks of companies with low operating profitability. The idea behind RMW is that historically, companies with high profitability tend to outperform companies with low profitability.
- Formula: `RMW = Robust - Weak` 

     where `Robust`: return on companies with high operating profitability, `Weak`: return on companies with low operating profitability
5. **CMA**: investment premium, which is the difference in returns between stocks of companies that invest conservatively and stocks of companies that invest aggressively. The idea behind CMA is that historically, companies that invest conservatively tend to outperform companies that invest aggressively.
- Formula: `CMA = Conservative - Aggressive` 

     where `Conservative`: return on companies that invest conservatively, `Aggressive`: return on companies that invest aggressively
- Formula in FF:  is the average return on the two conservative investment portfolios minus the average return on the two aggressive investment portfolios
 
 `CMA = 1/2 * (Small Conservative + Big Conservative) - 1/2 * (Small Aggressive + Big Aggressive)`
 

In [6]:
ff = pd.read_csv('data/fama_french.csv')  #unit of columns is %
print(ff.head(10))

       Date  Mkt-RF   SMB   HML   RMW   CMA     RF
0  19630701   -0.67  0.02 -0.35  0.03  0.13  0.012
1  19630702    0.79 -0.28  0.28 -0.08 -0.21  0.012
2  19630703    0.63 -0.18 -0.10  0.13 -0.25  0.012
3  19630705    0.40  0.09 -0.28  0.07 -0.30  0.012
4  19630708   -0.63  0.07 -0.20 -0.27  0.06  0.012
5  19630709    0.45  0.00  0.09  0.15 -0.01  0.012
6  19630710   -0.18  0.20  0.00  0.05 -0.09  0.012
7  19630711   -0.16  0.12 -0.29 -0.03  0.07  0.012
8  19630712   -0.12  0.01 -0.09  0.09  0.08  0.012
9  19630715   -0.62  0.07 -0.03  0.15 -0.10  0.012


In [7]:
#convert date to datetime
ff['Date'] = pd.to_datetime(ff['Date'], format='%Y%m%d')

ff.head()

,Date,Mkt-RF,SMB,HML,RMW,CMA,RF
0,1963-07-01,-0.67,0.02,-0.35,0.03,0.13,0.012
1,1963-07-02,0.79,-0.28,0.28,-0.08,-0.21,0.012
2,1963-07-03,0.63,-0.18,-0.10,0.13,-0.25,0.012
3,1963-07-05,0.40,0.09,-0.28,0.07,-0.30,0.012
4,1963-07-08,-0.63,0.07,-0.20,-0.27,0.06,0.012


In [8]:
#only lower case column "Date"
all_cols = ff.columns
new_cols = ["date"] + list(all_cols[1:])
ff.columns = new_cols
ff.head()

,date,Mkt-RF,SMB,HML,RMW,CMA,RF
0,1963-07-01,-0.67,0.02,-0.35,0.03,0.13,0.012
1,1963-07-02,0.79,-0.28,0.28,-0.08,-0.21,0.012
2,1963-07-03,0.63,-0.18,-0.10,0.13,-0.25,0.012
3,1963-07-05,0.40,0.09,-0.28,0.07,-0.30,0.012
4,1963-07-08,-0.63,0.07,-0.20,-0.27,0.06,0.012


In [9]:
ff.to_csv('data/formatted_datasets/fama_french.csv')

## ADS Index

The **average value of the ADS index is zero**, A value of zero indicates average economic conditions, meaning the economy is performing as expected without significant deviations. Values greater than zero represent better-than-average economic conditions. Conversely, values below zero indicate worse-than-average conditions.

In [10]:
ads = pd.read_csv('data/ADS_index.csv')
ads.head()


,Date,ADS_Index
0,1960:03:01,-0.578899
1,1960:03:02,-0.627759
2,1960:03:03,-0.673502
3,1960:03:04,-0.716147
4,1960:03:05,-0.755708


In [11]:
#change the first column name to "date"
ads.columns = ["date", "ads_index"]
ads.head()

,date,ads_index
0,1960:03:01,-0.578899
1,1960:03:02,-0.627759
2,1960:03:03,-0.673502
3,1960:03:04,-0.716147
4,1960:03:05,-0.755708


In [12]:
ads.dtypes

date          object
ads_index    float64
dtype: object

In [13]:
# change : to - in date column
ads["date"] = ads["date"].str.replace(":", "-")
ads["date"] = pd.to_datetime(ads["date"], format='%Y-%m-%d')
ads.head()

,date,ads_index
0,1960-03-01,-0.578899
1,1960-03-02,-0.627759
2,1960-03-03,-0.673502
3,1960-03-04,-0.716147
4,1960-03-05,-0.755708


In [14]:
#save to csv
ads.to_csv('data/formatted_datasets/ads_index.csv')

## SP500

In [15]:
api_key = '74a4c86d8f52f8875f7e465e42f8e5de'

fred = Fred(api_key=api_key)
data = fred.get_series('SP500')
data.head()

2014-10-27    1961.63
2014-10-28    1985.05
2014-10-29    1982.30
2014-10-30    1994.65
2014-10-31    2018.05
dtype: float64

In [16]:
# Define the FRED API endpoint
base_url = 'https://api.stlouisfed.org/fred/'

'''
Get observation data from the FRED API
'''

# Assign endpoint
obs_endpoint = 'series/observations'

# Assign parameters
series_id_list = ['SP500']
start_date = '1990-01-01'
end_date = '2024-10-18'
ts_frequency = 'd'
ts_units = 'pc1'

df = pd.DataFrame()

for series_id in series_id_list:
    obs_params = {
        'series_id': series_id,
        'api_key': api_key,
        'file_type': 'json',
        'observation_start': start_date,
        'observation_end': end_date,
        # 'frequency': ts_frequency
        # 'units': ts_units
    }

    # Make request to FRED API
    response = requests.get(base_url + obs_endpoint, params=obs_params)

    # Format data
    if response.status_code == 200:
        res_data = response.json()
        obs_data = pd.DataFrame(res_data['observations']) 
        obs_data = obs_data[['date', 'value']]
        obs_data['date'] = pd.to_datetime(obs_data['date'])
        obs_data['value'] = pd.to_numeric(obs_data['value'], errors='coerce')
        df = pd.concat([df, obs_data], axis=1)
    else:
        print('Failed to retrieve data. Status code:', response.status_code)
print(df.head())
print(df.shape)


        date    value
0 2014-10-27  1961.63
1 2014-10-28  1985.05
2 2014-10-29  1982.30
3 2014-10-30  1994.65
4 2014-10-31  2018.05
(2605, 2)


In [17]:
df.rename(columns={'value': 'SP500'}, inplace=True)

In [18]:
#save to csv
df.to_csv('data/formatted_datasets/sp500.csv')

## FRED

In [19]:
#read all the files in the directory data/fred_variables
files = glob.glob('data/fred_variables/*.csv')
files

['data/fred_variables/DGS10.csv',
 'data/fred_variables/RIFSPPFAAD90NB.csv',
 'data/fred_variables/DGS1.csv',
 'data/fred_variables/CBETHUSD.csv',
 'data/fred_variables/DBAA.csv',
 'data/fred_variables/USRECDP.csv',
 'data/fred_variables/USRECD.csv',
 'data/fred_variables/DCPN3M.csv',
 'data/fred_variables/DCPF1M.csv',
 'data/fred_variables/DAAA.csv',
 'data/fred_variables/T10Y3M.csv',
 'data/fred_variables/BAMLH0A0HYM2.csv',
 'data/fred_variables/T10YIE.csv',
 'data/fred_variables/DHHNGSP.csv',
 'data/fred_variables/AMERIBOR.csv',
 'data/fred_variables/USRECDM.csv',
 'data/fred_variables/CBBTCUSD.csv',
 'data/fred_variables/DEXUSUK.csv',
 'data/fred_variables/DCOILWTICO.csv',
 'data/fred_variables/VIXCLS.csv',
 'data/fred_variables/DEXJPUS.csv',
 'data/fred_variables/DEXUSEU.csv',
 'data/fred_variables/T5YIE.csv',
 'data/fred_variables/OBMMIJUMBO30YF.csv',
 'data/fred_variables/NIKKEI225.csv',
 'data/fred_variables/BAMLH0A0HYM2EY.csv',
 'data/fred_variables/DCOILBRENTEU.csv']

In [20]:
res = pd.read_csv(files[0])
res['DATE'] = pd.to_datetime(res['DATE'], format='%Y-%m-%d')
for file in files[1:]:
    #convert DATE to date
    df = pd.read_csv(file)
    df['DATE'] = pd.to_datetime(df['DATE'], format='%Y-%m-%d')
    res = pd.merge(res, df, on='DATE', how='outer')
res = res.sort_values('DATE')
res["DATE"] = res["DATE"].dt.strftime('%Y-%m-%d')
print(res)

             DATE DGS10 RIFSPPFAAD90NB  DGS1 CBETHUSD  DBAA  USRECDP  USRECD  \
17265  1854-12-01   NaN            NaN   NaN      NaN   NaN      0.0     1.0   
17266  1854-12-02   NaN            NaN   NaN      NaN   NaN      0.0     1.0   
17267  1854-12-03   NaN            NaN   NaN      NaN   NaN      0.0     1.0   
17268  1854-12-04   NaN            NaN   NaN      NaN   NaN      0.0     1.0   
17269  1854-12-05   NaN            NaN   NaN      NaN   NaN      0.0     1.0   
...           ...   ...            ...   ...      ...   ...      ...     ...   
16383  2024-10-18  4.08            NaN  4.19  2647.97  5.61      0.0     0.0   
17262  2024-10-19   NaN            NaN   NaN  2746.22   NaN      0.0     0.0   
17263  2024-10-20   NaN            NaN   NaN  2665.35   NaN      0.0     0.0   
17264  2024-10-21   NaN            NaN   NaN  2667.59  5.73      NaN     NaN   
62052  2024-10-22   NaN            NaN   NaN      NaN   NaN      NaN     NaN   

      DCPN3M DCPF1M  ... DEXUSUK DCOILW

In [21]:
#only lower case column "Date"
all_cols = res.columns
new_cols = ["date"] + list(all_cols[1:])
res.columns = new_cols
res.head()

,date,DGS10,RIFSPPFAAD90NB,DGS1,CBETHUSD,DBAA,USRECDP,USRECD,DCPN3M,DCPF1M,...,DEXUSUK,DCOILWTICO,VIXCLS,DEXJPUS,DEXUSEU,T5YIE,OBMMIJUMBO30YF,NIKKEI225,BAMLH0A0HYM2EY,DCOILBRENTEU
17265,1854-12-01,NaN,NaN,NaN,NaN,NaN,0.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17266,1854-12-02,NaN,NaN,NaN,NaN,NaN,0.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17267,1854-12-03,NaN,NaN,NaN,NaN,NaN,0.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17268,1854-12-04,NaN,NaN,NaN,NaN,NaN,0.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17269,1854-12-05,NaN,NaN,NaN,NaN,NaN,0.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
res.to_csv('data/formatted_datasets/fred_variables.csv', index=False)

# Merge all the datasets

In [23]:
files = glob.glob('data/formatted_datasets/*.csv')
files

['data/formatted_datasets/fama_french.csv',
 'data/formatted_datasets/fred_variables.csv',
 'data/formatted_datasets/ads_index.csv',
 'data/formatted_datasets/historical_data.csv',
 'data/formatted_datasets/sp500.csv']

In [24]:
res = pd.read_csv(files[0])
res.head(2)

,Unnamed: 0,date,Mkt-RF,SMB,HML,RMW,CMA,RF
0,0,1963-07-01,-0.67,0.02,-0.35,0.03,0.13,0.012
1,1,1963-07-02,0.79,-0.28,0.28,-0.08,-0.21,0.012


In [25]:
res = pd.read_csv(files[0])
res['date'] = pd.to_datetime(res['date'], format='%Y-%m-%d')
for file in files[1:]:
    #convert DATE to date
    df = pd.read_csv(file)
    df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
    res = pd.merge(res, df, on='date', how='outer')
res = res.sort_values('date')
res["date"] = res["date"].dt.strftime('%Y-%m-%d')
print(res)

/var/folders/24/k3t__8rn5j7f10302wn8dh240000gn/T/ipykernel_44244/2242375558.py:5: DtypeWarning: Columns (1,2,3,4,5,8,9,10,11,12,13,14,15,17,18,19,20,21,22,23,24,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


      Unnamed: 0_x        date  Mkt-RF  SMB  HML  RMW  CMA  RF DGS10  \
15397          NaN  1854-12-01     NaN  NaN  NaN  NaN  NaN NaN   NaN   
15398          NaN  1854-12-02     NaN  NaN  NaN  NaN  NaN NaN   NaN   
15399          NaN  1854-12-03     NaN  NaN  NaN  NaN  NaN NaN   NaN   
15400          NaN  1854-12-04     NaN  NaN  NaN  NaN  NaN NaN   NaN   
15401          NaN  1854-12-05     NaN  NaN  NaN  NaN  NaN NaN   NaN   
...            ...         ...     ...  ...  ...  ...  ...  ..   ...   
62051          NaN  2024-10-21     NaN  NaN  NaN  NaN  NaN NaN   NaN   
62052          NaN  2024-10-22     NaN  NaN  NaN  NaN  NaN NaN   NaN   
62053          NaN  2024-10-23     NaN  NaN  NaN  NaN  NaN NaN   NaN   
62054          NaN  2024-10-24     NaN  NaN  NaN  NaN  NaN NaN   NaN   
62055          NaN  2024-10-25     NaN  NaN  NaN  NaN  NaN NaN   NaN   

      RIFSPPFAAD90NB  ... Unnamed: 0_x        open        high         low  \
15397            NaN  ...          NaN         NaN       

/var/folders/24/k3t__8rn5j7f10302wn8dh240000gn/T/ipykernel_44244/2242375558.py:7: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Unnamed: 0_x'} in the result is deprecated and will raise a MergeError in a future version.
  res = pd.merge(res, df, on='date', how='outer')


In [26]:
res.columns

Index(['Unnamed: 0_x', 'date', 'Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF',
       'DGS10', 'RIFSPPFAAD90NB', 'DGS1', 'CBETHUSD', 'DBAA', 'USRECDP',
       'USRECD', 'DCPN3M', 'DCPF1M', 'DAAA', 'T10Y3M', 'BAMLH0A0HYM2',
       'T10YIE', 'DHHNGSP', 'AMERIBOR', 'USRECDM', 'CBBTCUSD', 'DEXUSUK',
       'DCOILWTICO', 'VIXCLS', 'DEXJPUS', 'DEXUSEU', 'T5YIE', 'OBMMIJUMBO30YF',
       'NIKKEI225', 'BAMLH0A0HYM2EY', 'DCOILBRENTEU', 'Unnamed: 0_y',
       'ads_index', 'Unnamed: 0_x', 'open', 'high', 'low', 'close', 'volume',
       'dividends', 'stock_splits', 'Unnamed: 0_y', 'SP500'],
      dtype='object')

In [27]:
# drop columns with name contains "Unnamed"
res = res.loc[:, ~res.columns.str.contains('^Unnamed')]

In [28]:
res.head(2)

,date,Mkt-RF,SMB,HML,RMW,CMA,RF,DGS10,RIFSPPFAAD90NB,DGS1,...,DCOILBRENTEU,ads_index,open,high,low,close,volume,dividends,stock_splits,SP500
15397,1854-12-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15398,1854-12-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
res.columns

Index(['date', 'Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF', 'DGS10',
       'RIFSPPFAAD90NB', 'DGS1', 'CBETHUSD', 'DBAA', 'USRECDP', 'USRECD',
       'DCPN3M', 'DCPF1M', 'DAAA', 'T10Y3M', 'BAMLH0A0HYM2', 'T10YIE',
       'DHHNGSP', 'AMERIBOR', 'USRECDM', 'CBBTCUSD', 'DEXUSUK', 'DCOILWTICO',
       'VIXCLS', 'DEXJPUS', 'DEXUSEU', 'T5YIE', 'OBMMIJUMBO30YF', 'NIKKEI225',
       'BAMLH0A0HYM2EY', 'DCOILBRENTEU', 'ads_index', 'open', 'high', 'low',
       'close', 'volume', 'dividends', 'stock_splits', 'SP500'],
      dtype='object')

In [30]:
#save to csv
res.to_csv('data/final_dataset.csv', index=False)